In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import time

In [ ]:
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 3

def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.convert('RGB')
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = np.array(img).astype(np.float32)
    img_array = (img_array - 127.5) / 127.5 # Normalize to [-1,1]
    return img_array
data_dirs = [
    '/kaggle/input/cubicasa5k/cubicasa5k/cubicasa5k/high_quality',
]

all_image_paths = []
for d in data_dirs:
    for root, dirs, files in os.walk(d):
        for filename in files:
            if filename.lower().endswith('.png'):
                full_path = os.path.join(root, filename)
                if os.path.exists(full_path):
                    all_image_paths.append(full_path)

print("Total valid images found:", len(all_image_paths))

def process_path(file_path):
    img = tf.numpy_function(func=load_and_preprocess_image, inp=[file_path], Tout=tf.float32)
    img.set_shape([IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])
    return img

dataset = tf.data.Dataset.from_tensor_slices(all_image_paths)
dataset = dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
BATCH_SIZE = 64
dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

def make_discriminator_model():
    model = tf.keras.Sequential(name="discriminator")
    model.add(tf.keras.layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', 
                                     input_shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS],
                                     kernel_initializer=initializer))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same',
                                     kernel_initializer=initializer))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same',
                                     kernel_initializer=initializer))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same',
                                     kernel_initializer=initializer))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same',
                                     kernel_initializer=initializer))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

def make_generator_model():
    model = tf.keras.Sequential(name="generator")
    n_nodes = 256 * 4 * 4
    model.add(tf.keras.layers.Dense(n_nodes, input_dim=100, kernel_initializer=initializer))
    model.add(tf.keras.layers.Reshape((4, 4, 256)))
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', 
                                             activation='relu', kernel_initializer=initializer))
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', 
                                             activation='relu', kernel_initializer=initializer))
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', 
                                             activation='relu', kernel_initializer=initializer))
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', 
                                             activation='relu', kernel_initializer=initializer))
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', 
                                             activation='relu', kernel_initializer=initializer))
    model.add(tf.keras.layers.Conv2D(3, (4, 4), activation='tanh', padding='same'))
    
    return model

generator = make_generator_model()
discriminator = make_discriminator_model()


In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False) 

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [ ]:
def generate_latent_points(latent_dim, n_samples):
    return tf.random.normal([n_samples, latent_dim])

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    
    # Scale images from [-1,1] to [0,1]
    predictions = (predictions + 1) / 2.0
    
    fig = plt.figure(figsize=(4, 4))
    
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')
    
    plt.suptitle(f"Epoch {epoch}")
    
    # Save the image
    filename = f"generated_epoch_{epoch}.png"
    plt.savefig(filename)
    plt.close()


In [ ]:
@tf.function
def train_step(images):
    noise = generate_latent_points(100, BATCH_SIZE)
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss


In [ ]:
def train(dataset, epochs):
    seed = generate_latent_points(100, 16)
    
    for epoch in range(epochs):
        start = time.time()
        print(f"Starting epoch {epoch+1}/{epochs}...")
        
        batch_count = 0
        for image_batch in dataset:
            g_loss, d_loss = train_step(image_batch)
            batch_count += 1
            
            if batch_count % 10 == 0:
                print(f">Epoch {epoch+1}, Batch {batch_count}, D_loss={d_loss:.3f}, G_loss={g_loss:.3f}")
        
        generate_and_save_images(generator, epoch+1, seed)
        
        if (epoch + 1) % 10 == 0:
            generator.save(f'generator_model_epoch_{epoch+1}.h5')
            discriminator.save(f'discriminator_model_epoch_{epoch+1}.h5')
        
        print(f"Time for epoch {epoch+1} is {time.time()-start:.2f} sec")


In [ ]:
EPOCHS = 100
print("Starting training...")
train(dataset, EPOCHS)

In [ ]:
def test_generator(generator_model, num_samples=16):
    print("Generating test samples...")
    
    noise = tf.random.normal([num_samples, 100])
    generated_images = generator_model(noise, training=False)
    generated_images = (generated_images + 1) / 2.0
    
    fig = plt.figure(figsize=(4, 4))
    for i in range(num_samples):
        plt.subplot(4, 4, i+1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig("test_generated_samples.png")
    plt.show()
    
    print("Test samples generated and saved as 'test_generated_samples.png'")

test_generator(generator)